In [1]:
import gym
import random
import matplotlib.pyplot as plt

# 1. Test Random Environment with OpenAI Gym

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:0 Score:135.0
Episode:1 Score:155.0
Episode:2 Score:105.0
Episode:3 Score:210.0
Episode:4 Score:105.0


# 2. Create a Deep learning Model with Keras

In [5]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height,width,channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.__len__=actions
    return model

In [7]:
model = build_model(height, width, channels, actions)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 250)               128250    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# 3. Build Agent with Keras-RL

In [30]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [35]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [36]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


C:\Users\mclea\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  603/10000: episode: 1, duration: 3.609s, episode steps: 603, steps per second: 167, episode reward: 105.000, mean reward:  0.174 [ 0.000, 25.000], mean action: 2.517 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\mclea\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1129/10000: episode: 2, duration: 17.507s, episode steps: 526, steps per second:  30, episode reward: 30.000, mean reward:  0.057 [ 0.000, 15.000], mean action: 2.563 [0.000, 5.000],  loss: 18.005289, mean_q: 6.854127, mean_eps: 0.904195
 1866/10000: episode: 3, duration: 73.146s, episode steps: 737, steps per second:  10, episode reward: 135.000, mean reward:  0.183 [ 0.000, 25.000], mean action: 2.498 [0.000, 5.000],  loss: 0.658710, mean_q: 5.172617, mean_eps: 0.865270
 2806/10000: episode: 4, duration: 95.532s, episode steps: 940, steps per second:  10, episode reward: 170.000, mean reward:  0.181 [ 0.000, 30.000], mean action: 2.296 [0.000, 5.000],  loss: 0.811811, mean_q: 5.534914, mean_eps: 0.789805
 3685/10000: episode: 5, duration: 88.947s, episode steps: 879, steps per second:  10, episode reward: 285.000, mean reward:  0.324 [ 0.000, 30.000], mean action: 2.268 [0.000, 5.000],  loss: 0.771942, mean_q: 4.554386, mean_eps: 0.707950
 4773/10000: episode: 6, duration: 108.839s,

In [37]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history(["episode_reward"])))

Testing for 10 episodes ...
Episode 1: reward: 110.000, steps: 728
Episode 2: reward: 335.000, steps: 798
Episode 3: reward: 120.000, steps: 818
Episode 4: reward: 80.000, steps: 503
Episode 5: reward: 150.000, steps: 958
Episode 6: reward: 135.000, steps: 770
Episode 7: reward: 135.000, steps: 634
Episode 8: reward: 105.000, steps: 777
Episode 9: reward: 105.000, steps: 515
Episode 10: reward: 85.000, steps: 610


TypeError: 'dict' object is not callable